In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys

# making sure the dl4mt module is on the path 
# -- this path depends upon the location where the notebook is running
# here it is assumed to be the day1/ directory, change it to point to the location of the dl4mt/ dir if necessary 
sys.path.append('../..')

In [2]:
%%writefile ../../dl4mt/mlp.py
# Uncomment to save this cell to file in order to import it in later notebooks

# construct an MLP graph using theano, then train and evaluate using static features
import numpy
import theano
import theano.tensor as T

# this is our logistic regression class, created and saved in the LogisticRegression notebook
from dl4mt.logistic_regression import LogisticRegression

class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, W=None, b=None,
                 activation=T.tanh):
        """
        Typical hidden layer of a MLP: units are fully-connected and have
        sigmoidal activation function. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.dmatrix
        :param input: a symbolic tensor of shape (n_examples, n_in)

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: theano.Op or function
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        self.input = input

        # `W` is initialized with `W_values` which is uniformely sampled
        # from sqrt(-6./(n_in+n_hidden)) and sqrt(6./(n_in+n_hidden))
        # for tanh activation function
        # the output of uniform if converted using asarray to dtype
        # theano.config.floatX so that the code is runable on GPU
        # Note : optimal initialization of weights is dependent on the
        #        activation function used (among other things).
        #        For example, results presented in [Xavier10] suggest that you
        #        should use 4 times larger initial weights for sigmoid
        #        compared to tanh
        #        We have no info for other function, so we use the same as
        #        tanh.
        if W is None:
            W_values = numpy.asarray(
                rng.uniform(
                    low=-numpy.sqrt(6. / (n_in + n_out)),
                    high=numpy.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                dtype=theano.config.floatX
            )
            if activation == theano.tensor.nnet.sigmoid:
                W_values *= 4

            W = theano.shared(value=W_values, name='W', borrow=True)

        if b is None:
            b_values = numpy.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b', borrow=True)

        self.W = W
        self.b = b

        lin_output = T.dot(input, self.W) + self.b
        self.output = (
            lin_output if activation is None
            else activation(lin_output)
        )
        # parameters of the model
        self.params = [self.W, self.b]


class MLP(object):
    """Multi-Layer Perceptron Class

    A multilayer perceptron is a feedforward artificial neural network model
    that has one layer or more of hidden units and nonlinear activations.
    Intermediate layers usually have as activation function tanh or the
    sigmoid function (defined here by a ``HiddenLayer`` class)  while the
    top layer is a softmax layer (defined here by a ``LogisticRegression``
    class).
    """

    def __init__(self, rng, input, n_in, n_hidden, n_out):
        """Initialize the parameters for the multilayer perceptron

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.TensorType
        :param input: symbolic variable that describes the input of the
        architecture (one minibatch)

        :type n_in: int
        :param n_in: number of input units, the dimension of the space in
        which the datapoints lie

        :type n_hidden: int
        :param n_hidden: number of hidden units

        :type n_out: int
        :param n_out: number of output units, the dimension of the space in
        which the labels lie

        """

        # Since we are dealing with a one hidden layer MLP, this will translate
        # into a HiddenLayer with a tanh activation function connected to the
        # LogisticRegression layer; the activation function can be replaced by
        # sigmoid or any other nonlinear function
        self.hiddenLayer = HiddenLayer(
            rng=rng,
            input=input,
            n_in=n_in,
            n_out=n_hidden,
            activation=T.tanh
        )

        # The logistic regression layer gets as input the hidden units
        # of the hidden layer
        self.logRegressionLayer = LogisticRegression(
            input=self.hiddenLayer.output,
            n_in=n_hidden,
            n_out=n_out
        )
        
        # alias to logRegressionLayer.y_pred to provide a consistent interface to our models
        self.y_pred = self.logRegressionLayer.y_pred

        # L1 norm; one regularization option is to enforce L1 norm to be small
        self.L1 = (
            abs(self.hiddenLayer.W).sum()
            + abs(self.logRegressionLayer.W).sum()
        )

        # square of L2 norm ; one regularization option is to enforce
        # square of L2 norm to be small
        self.L2_sqr = (
            (self.hiddenLayer.W ** 2).sum()
            + (self.logRegressionLayer.W ** 2).sum()
        )

        # negative log likelihood of the MLP is given by the negative
        # log likelihood of the output of the model, computed in the
        # logistic regression layer
        self.negative_log_likelihood = (
            self.logRegressionLayer.negative_log_likelihood
        )
        # same holds for the function computing the number of errors
        self.errors = self.logRegressionLayer.errors

        # the parameters of the model are the parameters of the two layer it is
        # made out of
        self.params = self.hiddenLayer.params + self.logRegressionLayer.params

        # keep track of model input -- this will be useful for creating theano functions
        self.input = input

Overwriting ../../dl4mt/mlp.py


In [3]:
import numpy

import theano
import theano.tensor as T
import timeit

from dl4mt.mlp import MLP


def initialize_mlp(train_dataset, dev_dataset, learning_rate=0.01, L1_reg=0.00, L2_reg=0.0001, 
                   batch_size=10, n_hidden=500):
    
    """Intialize the functions and parameters needed to train a multilayer perceptron

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
    gradient

    :type L1_reg: float
    :param L1_reg: L1-norm's weight when added to the cost (see
    regularization)

    :type L2_reg: float
    :param L2_reg: L2-norm's weight when added to the cost (see
    regularization)

    :type training_epochs: int
    :param training_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz


   """

    train_set_x, train_set_y = train_dataset
    valid_set_x, valid_set_y = dev_dataset

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch
    x = T.matrix('x')  # the data is presented as rasterized images
    y = T.ivector('y')  # the labels are presented as 1D vector of
                        # [int] labels

    rng = numpy.random.RandomState(1234)
    
    # TODO: move these into the parameterization of the training function
    n_in = train_set_x.get_value().shape[1]
    n_out = 12
    
    # construct the MLP class
    classifier = MLP(
        rng=rng,
        input=x,
        n_in=n_in,
        n_hidden=n_hidden,
        n_out=n_out
    )

    # the cost we minimize during training is the negative log likelihood of
    # the model plus the regularization terms (L1 and L2); cost is expressed
    # here symbolically
    cost = (
        classifier.negative_log_likelihood(y)
        + L1_reg * classifier.L1
        + L2_reg * classifier.L2_sqr
    )

    validate_model_func = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size:(index + 1) * batch_size],
            y: valid_set_y[index * batch_size:(index + 1) * batch_size]
        }
    )

    # compute the gradient of cost with respect to theta (sotred in params)
    # the resulting gradients will be stored in a list gparams
    gparams = [T.grad(cost, param) for param in classifier.params]

    # specify how to update the parameters of the model as a list of
    # (variable, update expression) pairs
    updates = [
        (param, param - learning_rate * gparam)
        for param, gparam in zip(classifier.params, gparams)
    ]

    # compiling a Theano function `train_model` that returns the cost, but
    # in the same time updates the parameters of the model based on the rules
    # defined in `updates`
    train_model_func = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    return (classifier, train_model_func, validate_model_func, n_train_batches, n_valid_batches)

In [4]:
# Challenge: implement multiple hidden layers to make this a true "Multilayer Perceptron", 
# how does the performance of the model change?

In [5]:
# TODO: allow user to load another distributed representation for comparison, such as PCA
import os

from dl4mt.datasets import prep_dataset
from dl4mt.training import train_model

# location of our datasets
DATASET_LOCATION = '../../datasets/'

# the pos dataset consists of windows around words
POS_DATASET_NAME = 'brown_pos_dataset.hdf5'
POS_DATASET_PATH = os.path.join(DATASET_LOCATION, POS_DATASET_NAME)
WORD_BY_WORD_MATRIX = 'brown.word-by-word.normalized.npy'
VECTOR_INDEX_PATH = os.path.join(DATASET_LOCATION, WORD_BY_WORD_MATRIX)

CUTOFF = 10000
  
train_dataset = prep_dataset(POS_DATASET_PATH, VECTOR_INDEX_PATH, which_sets=['train'], cutoff=CUTOFF)
dev_dataset = prep_dataset(POS_DATASET_PATH, VECTOR_INDEX_PATH, which_sets=['dev'], cutoff=CUTOFF)

# train the MLP
initialization_data = initialize_mlp(train_dataset, dev_dataset, learning_rate=0.01, batch_size=50)

classifier, train_model_func, validate_model_func, n_train_batches, n_valid_batches = initialization_data


# load the training function and train the LR model 
# TODO: -- train_model should return the validation error as a list, then we can plot it
# in general, train_model should return any useful information about the training process
train_model(train_model_func, n_train_batches, validate_model=validate_model_func,
            n_valid_batches=n_valid_batches, training_epochs=100)

epoch 1, minibatch 200/200, validation error 50.750000 %
epoch 1, training cost 1.70
epoch 2, minibatch 200/200, validation error 42.350000 %
epoch 2, training cost 1.25
epoch 3, minibatch 200/200, validation error 37.200000 %
epoch 3, training cost 1.05
epoch 4, minibatch 200/200, validation error 33.380000 %
epoch 4, training cost 0.92
epoch 5, minibatch 200/200, validation error 30.230000 %
epoch 5, training cost 0.83
epoch 6, minibatch 200/200, validation error 27.990000 %
epoch 6, training cost 0.76
epoch 7, minibatch 200/200, validation error 26.290000 %
epoch 7, training cost 0.71
epoch 8, minibatch 200/200, validation error 25.140000 %
epoch 8, training cost 0.67
epoch 9, minibatch 200/200, validation error 24.290000 %
epoch 9, training cost 0.64
epoch 10, minibatch 200/200, validation error 23.710000 %
epoch 10, training cost 0.61
epoch 11, minibatch 200/200, validation error 23.120000 %
epoch 11, training cost 0.58
epoch 12, minibatch 200/200, validation error 22.810000 %
epo

In [7]:
from dl4mt.predict import predict

test_dataset = prep_dataset(POS_DATASET_PATH, VECTOR_INDEX_PATH, which_sets=['test'], cutoff=CUTOFF, cast_y=False)

test_X, test_y = test_dataset
predictions = predict(classifier, test_X.get_value())
predictions
zip(predictions, list(test_y.get_value()))

[(1, 1.0),
 (1, 1.0),
 (2, 2.0),
 (1, 1.0),
 (0, 7.0),
 (6, 6.0),
 (1, 7.0),
 (2, 2.0),
 (4, 4.0),
 (1, 1.0),
 (2, 2.0),
 (1, 7.0),
 (1, 11.0),
 (1, 1.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (4, 4.0),
 (1, 1.0),
 (11, 11.0),
 (1, 11.0),
 (1, 1.0),
 (6, 6.0),
 (7, 7.0),
 (4, 4.0),
 (5, 5.0),
 (11, 11.0),
 (1, 1.0),
 (5, 5.0),
 (5, 5.0),
 (10, 10.0),
 (1, 1.0),
 (1, 1.0),
 (1, 1.0),
 (1, 1.0),
 (1, 1.0),
 (2, 2.0),
 (1, 1.0),
 (7, 7.0),
 (4, 0.0),
 (5, 5.0),
 (10, 10.0),
 (3, 3.0),
 (7, 7.0),
 (7, 7.0),
 (4, 4.0),
 (1, 1.0),
 (5, 5.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (0, 0.0),
 (7, 7.0),
 (0, 0.0),
 (11, 11.0),
 (5, 5.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (7, 7.0),
 (2, 2.0),
 (1, 1.0),
 (1, 0.0),
 (3, 3.0),
 (7, 7.0),
 (7, 7.0),
 (6, 6.0),
 (7, 7.0),
 (11, 11.0),
 (1, 1.0),
 (2, 2.0),
 (11, 11.0),
 (1, 1.0),
 (5, 5.0),
 (1, 0.0),
 (5, 5.0),
 (6, 6.0),
 (7, 7.0),
 (11, 11.0),
 (1, 1.0),
 (5, 5.0),
 (1, 1.0),
 (7, 7.0),
 (4, 4.0),
 (11, 7.0),
 (1, 1.0),
 (

In [8]:
# Quick Evaluation

sum([y==p for y,p in zip(predictions, list(test_y.get_value()))]) / float(len(predictions))

0.87257132456659237

In [18]:
# Challenge: change the MLP class to allow adding multiple hidden layers 
# Challenge: compare performance with different kinds of regularization
